# Aurite Agents Tutorial 2: Giving Agents Tools with MCP Servers

Welcome to the second tutorial in the Aurite Agents series! In the previous tutorial, we learned how to create and configure a basic agent. Now, we'll explore one of the most powerful features of the framework: giving agents **Tools** to interact with the outside world.

In Aurite, tools are provided by **Model Context Protocol (MCP) Servers**. These are external programs that can do anything from searching the web to analyzing data. By connecting an agent to an MCP Server, you give it access to all the tools that server provides (you can also exclude specific tools that you don't want agents to access when you are adding new mcp server configurations).

### What You'll Learn

In this notebook, you will learn:

1.  **What an MCP Server is** and how to configure one using a `ClientConfig` object.
2.  How to connect to a tool server that runs on the web (using the `http_stream` transport), which is perfect for cloud environments like Google Colab.
3.  How to use the library of **pre-packaged tools** that comes built-in with the `aurite` package.

## Section 1: Setup

### Step 1.1: Install Dependencies

First, let's install the `aurite` package.

In [ ]:
%pip install aurite==0.3.15

Note: you may need to restart the kernel to use updated packages.


### Step 1.2: Configure API Keys

Like in our previous notebooks, we need to configure our API keys. The cell below will prompt you to enter your keys securely.

In [29]:
import os
from getpass import getpass

try:
  from google.colab import userdata #type: ignore
  # Fetch the secret value from Colab's secret manager and set it as an environment variable so your script can find it
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
  os.environ["SMITHERY_API_KEY"] = userdata.get('SMITHERY_API_KEY')
  os.environ["SMITHERY_PROFILE_ID"] = userdata.get('SMITHERY_PROFILE_ID')
except ImportError: # This is not a real error, just a check if running in Colab
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except Exception as e:
    print(f"Error fetching from Colab's secret manager: {e}")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
if "SMITHERY_API_KEY" not in os.environ:
    os.environ["SMITHERY_API_KEY"] = getpass("Enter your Smithery API key: ")
if "SMITHERY_PROFILE_ID" not in os.environ:
    os.environ["SMITHERY_PROFILE_ID"] = getpass("Enter your Smithery Profile ID: ")

### Step 1.3: HTML Display Helper

We'll use the same helper function from the previous tutorials to render the agent's output in a nice, readable format.

In [30]:
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""

  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\"{query}\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

### Step 1.4: Initialize Aurite

Finally, let's create and initialize the main Aurite application object. We will use this single instance for the entire notebook.

In [31]:
from aurite import Aurite

aurite = Aurite()

await aurite.initialize()

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


## Section 2: Manually Registering a Tool Server

To fully understand how tools work, we will first walk through all the steps of configuring, registering, and using a tool server manually.

### Step 2.1: Understanding `ClientConfig`

A `ClientConfig` is a configuration object (which can be defined in Python or JSON) that tells Aurite everything it needs to know to connect to and use an MCP Server. 

Let's look at the configuration for a real tool server from the packaged toolbox: `game_trends_mcp`. This server provides tools for getting data about video games.

```json
{
    "name": "game_trends_mcp",
    "transport_type": "http_stream",
    "http_endpoint": "https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile={SMITHERY_PROFILE_ID}&api_key={SMITHERY_API_KEY}",
    "capabilities": ["tools"]
}
```

**Key Fields:**
-   `name`: A unique name for the server.
-   `transport_type`: How to connect. We're using `"http_stream"`, which is perfect for cloud notebooks because it connects to a server already running on the internet.
-   `http_endpoint`: The URL of the server. Notice the `{SMITHERY_PROFILE_ID}` and `{SMITHERY_API_KEY}` placeholders—Aurite will automatically replace these with the environment variables you just set!
-   `capabilities`: What the server provides. In this case, `"tools"`.

> For a full breakdown of all configuration options, including other transport types like `stdio` for local scripts, see the **[Client Configurations Documentation](../components/mcp_server.md)**.

### Step 2.2: Define and Register the `ClientConfig`

Instead of using a JSON file, we can define the `ClientConfig` directly in our Python code and register it with our running `aurite` instance.

In [ ]:
from aurite.config.config_models import ClientConfig

game_trends_config = ClientConfig(
    name="game_trends_mcp",
    http_endpoint="https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile={SMITHERY_PROFILE_ID}&api_key={SMITHERY_API_KEY}",
    capabilities=["tools"]
)
await aurite.register_client(game_trends_config)
print(f"✅ Successfully registered ClientConfig for: {game_trends_config.name}")

INFO     [aurite.host.host] Attempting to dynamically register client: game_trends_mcp
INFO     [aurite.host.host] Client 'game_trends_mcp' dynamically registered and initialized successfully.


✅ Successfully registered ClientConfig for: game_trends_mcp


### Step 2.3: Define and Register an Agent

Now, let's create an agent and tell it that it's allowed to use the server we just configured. We do this by adding the server's `name` to the agent's `mcp_servers` list.

In [33]:
from aurite.config.config_models import AgentConfig

# Use the name of the ClientConfig we just created to register an agent with it
analyst_agent_config = AgentConfig(
    name="Game Trend Analyst",
    system_prompt="You are an expert analyst in the video game industry. Use the tools provided to answer questions about game trends.",
    mcp_servers=["game_trends_mcp"] # This links the agent to our ClientConfig
)

await aurite.register_agent(analyst_agent_config)

print(f"✅ Successfully Registered Agent: {analyst_agent_config.name}")

✅ Successfully Registered Agent: Game Trend Analyst


### Step 2.4: Run the Agent

Now for the fun part! Let's ask our agent a question that requires it to use the game trends tool.

In [34]:
user_query = "What are the top-selling games on Steam right now?"
print(f"Running agent with query: '{user_query}'...")

agent_result = await aurite.run_agent(
    agent_name="Game Trend Analyst",
    user_message=user_query
)

display_agent_response("Game Trend Analyst", user_query, agent_result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'Game Trend Analyst' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Game Trend Analyst'...


Running agent with query: 'What are the top-selling games on Steam right now?'...


INFO     [aurite.host.host] Executing tool 'get_steam_top_sellers' on client 'game_trends_mcp' for agent 'Game Trend Analyst'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Game Trend Analyst' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Game Trend Analyst</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"What are the top-selling games on Steam right now?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The top-selling games on Steam right now are:

1. **Dune: Awakening**
   - Release Date: Jun 10, 2025
   - Price: N/A
   - [Link to Store](https://store.steampowered.com/app/1172710/Dune_Awakening/?snr=1_7_7_7000_150_1)

2. **Counter-Strike 2**
   - Release Date: Aug 21, 2012
   - Price: N/A
   - [Link to Store](https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_7000_150_1)

3. **ELDEN RING NIGHTREIGN**
   - Release Date: May 29, 2025
   - Price: N/A
   - [Link to Store](https://store.steampowered.com/app/2622380/ELDEN_RING_NIGHTREIGN/?snr=1_7_7_7000_150_1)

4. **Steam Deck**
   - Release Date: Jan 17, 2025
   - Price: N/A
   - [Link to Store](https://store.steampowered.com/app/1675200/Steam_Deck/?snr=1_7_7_7000_150_1)

5. **DELTARUNE**
   - Release Date: Jun 4, 2025
   - Price: N/A
   - [Link to Store](https://store.steampowered.com/app/1671210/DELTARUNE/?snr=1_7_7_7000_150_1)

These are the current top 5 bestsellers on Steam, showcasing a mix of upcoming and existing titles.
      </div>
    </div>
  </div>
  

## Section 3: The Easy Way - Using the Built-in Toolbox

Manually defining the `ClientConfig` is great for understanding how things work, but Aurite provides a much easier way to use the dozens of tools included in its toolbox: **Just-in-Time (JIT) Registration**.

You don't need to define a `ClientConfig` for any packaged tool. You just need to reference it by name in your agent's `mcp_servers` list, and Aurite will find it and set it up for you automatically.

### Step 3.1: Define and Register a New Agent

Let's create a new agent to be a weather forecaster. We'll tell it to use the `national_weather_service` tool server. Notice that this time, we are **not** creating a `ClientConfig` for it. We just register the agent, and Aurite handles the rest.

In [35]:
weather_agent_config = AgentConfig(
    name="Weather Forecaster",
    system_prompt="You are a helpful weather assistant. Use the tools provided to get the weather forecast.",
    mcp_servers=["national_weather_service"] # Just reference the packaged tool by name!
)
await aurite.register_agent(weather_agent_config)
print(f"✅ Successfully Registered Agent: {weather_agent_config.name}")

INFO     [aurite.host.host] Attempting to dynamically register client: national_weather_service
INFO     [aurite.host.host] Client 'national_weather_service' dynamically registered and initialized successfully.


✅ Successfully Registered Agent: Weather Forecaster


### Step 3.2: Run the New Agent

Now let's run our new weather agent.

In [36]:
user_query_weather = "What's the weather forecast for New York, NY?"
print(f"Running agent with query: '{user_query_weather}'...")

agent_result_weather = await aurite.run_agent(
    agent_name="Weather Forecaster",
    user_message=user_query_weather
)

display_agent_response("Weather Forecaster", user_query_weather, agent_result_weather.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'Weather Forecaster' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Weather Forecaster'...


Running agent with query: 'What's the weather forecast for New York, NY?'...


INFO     [aurite.host.host] Executing tool 'get_weather_forecast' on client 'national_weather_service' for agent 'Weather Forecaster'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Weather Forecaster' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Weather Forecaster</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>"What's the weather forecast for New York, NY?"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        Here's the 7-day weather forecast for New York, NY:

### Tonight
- **Temperature:** 62°F, rising to around 64°F overnight.
- **Conditions:** Chance of rain showers, 45% chance of precipitation.
- **Wind:** East at 13 mph.

### Monday
- **Temperature:** 67°F, falling to around 65°F in the afternoon.
- **Conditions:** Chance of showers and thunderstorms, 40% chance of precipitation.
- **Wind:** East at 12 mph.

### Monday Night
- **Temperature:** 65°F, rising to around 67°F overnight.
- **Conditions:** Chance of rain showers, 51% chance of precipitation.
- **Wind:** East 6 to 10 mph.

### Tuesday
- **Temperature:** 73°F.
- **Conditions:** Showers and thunderstorms likely, 71% chance of precipitation.
- **Wind:** Southeast at 7 mph.

### Tuesday Night
- **Temperature:** 66°F.
- **Conditions:** Chance of showers and thunderstorms before 8pm, then partly cloudy, 43% chance of precipitation.
- **Wind:** West at 7 mph.

### Wednesday
- **Temperature:** 79°F.
- **Conditions:** Sunny.
- **Wind:** West 6 to 12 mph.

### Wednesday Night
- **Temperature:** 69°F.
- **Conditions:** Clear.
- **Wind:** West at 12 mph.

### Thursday
- **Temperature:** 84°F.
- **Conditions:** Sunny, 10% chance of precipitation.
- **Wind:** West at 12 mph.

### Thursday Night
- **Temperature:** 71°F.
- **Conditions:** Partly cloudy, 10% chance of precipitation.
- **Wind:** West 6 to 10 mph.

### Friday
- **Temperature:** 80°F.
- **Conditions:** Partly sunny then a slight chance of rain showers after 2pm, 19% chance of precipitation.
- **Wind:** Northwest at 8 mph.

### Friday Night
- **Temperature:** 67°F.
- **Conditions:** Chance of rain showers, 25% chance of precipitation.
- **Wind:** North at 8 mph.

### Saturday
- **Temperature:** 72°F.
- **Conditions:** Chance of rain showers before 2pm, then a chance of showers and thunderstorms, 37% chance of precipitation.
- **Wind:** East 8 to 12 mph.

### Saturday Night
- **Temperature:** 64°F.
- **Conditions:** Chance of showers and thunderstorms, 45% chance of precipitation.
- **Wind:** East at 12 mph.

### Sunday
- **Temperature:** 71°F.
- **Conditions:** Chance of rain showers, 39% chance of precipitation.
- **Wind:** East at 12 mph.

This forecast provides a detailed look at the expected weather conditions, including temperature, precipitation, and wind direction and speed for the coming week in New York, NY.
      </div>
    </div>
  </div>
  

### Step 3.3: How It Works

It just works! When you registered the `Weather Forecaster` agent, Aurite's `HostManager` saw that it needed an MCP server named `national_weather_service`. It searched the built-in toolbox, found the corresponding `ClientConfig`, loaded it, and automatically connected to the server using the Smithery API keys you provided earlier. 

This makes using the entire library of packaged tools incredibly simple.

## Section 4: Your Turn to Explore

Now that you know how to use packaged tools, you can experiment with the entire toolbox.

1.  Explore the full list of available tools in the **[Packaged MCP Servers Directory](../toolbox/mcp_server_directory.md)**.
2.  Click on a category to see the available servers and the tools they provide.
3.  Try creating a new agent that uses a different server, like `pubmed_mcp_server` (for medical research) or `appinsightmcp` (for app store data).

**Challenge:** Create a new agent named "Medical Researcher" that uses the `pubmed_mcp_server` and ask it to `"Search for recent articles about AI in medicine."`

## Conclusion

Congratulations! You've learned the fundamentals of giving agents tools in the Aurite framework. 

You now know:
-   How to define and register a `ClientConfig` to connect to any `http_stream` MCP server.
-   How to leverage the powerful Just-in-Time registration to use any of the dozens of tools in the built-in toolbox just by referencing their name.

This opens up a world of possibilities for creating highly capable and specialized agents.